In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from shapely import geometry
import pandas as pd

In [9]:
Polygon = geometry.polygon.Polygon
MultiPolygon = geometry.multipolygon.MultiPolygon

In [10]:
def parse(gdf, arg_i, arg_shape_id, arg_polygon_id):
    columns = ["geovertex_id", "shape_id", "polygon_id", "CName", "x_pos", "y_pos"]
    df = pd.DataFrame(columns=columns)

    counter = 0
    i = arg_i
    shape_id = arg_shape_id
    polygon_id = arg_polygon_id

    for index, row in gdf.iterrows():
        # for visualization only
        if counter%15==0:
            print(" {:.2f}% ".format(counter/len(gdf)*100), end="")
        else:
            print("#", end="")

        cur_shape = row["geometry"]
        cur_type = type(cur_shape)

        row = row.drop(["geometry"])
        cur_row = row
        cur_row["shape_id"] = shape_id
        shape_id += 1

        if cur_type == MultiPolygon:
            for polygon in cur_shape:
                cur_row["polygon_id"] = polygon_id
                polygon_id += 1
                for coords in polygon.exterior.coords:
                    cur_row["geovertex_id"] = i
                    i += 1
                    cur_row["x_pos"] = coords[0]
                    cur_row["y_pos"] = coords[1]
                    df = df.append(cur_row, ignore_index=True)
                    cur_row = row
        elif cur_type == Polygon:
            cur_row["polygon_id"] = polygon_id
            polygon_id += 1
            for coords in cur_shape.exterior.coords:
                cur_row["geovertex_id"] = i
                i += 1
                cur_row["x_pos"] = coords[0]
                cur_row["y_pos"] = coords[1]
                df = df.append(cur_row, ignore_index=True)
                cur_row = row
        counter += 1
        
    return df

In [ ]:
types = ["Ghost", "Gap"]
states = ["RI", "VA", "TX"]
next_i, next_shape, next_poly = 1495449, 959, 1940

for type_ in types:
    for state in states:
        file_name = state + "_" + type_ + "_" + "CName.json"
        gdf = gpd.read_file(file_name)

        print("\n\nStart", file_name)

        # parsing
        gdf_Parsed = parse(gdf, arg_i=next_i, arg_shape_id=next_shape, arg_polygon_id=next_poly)

        next_i = gdf_Parsed.iloc[-1]["geovertex_id"] + 1
        next_shape = gdf_Parsed.iloc[-1]["shape_id"] + 1
        next_poly = gdf_Parsed.iloc[-1]["polygon_id"] + 1
        print("\nLast row id: geo:{}, shape:{}, poly:{}".format(gdf_Parsed.iloc[-1]["geovertex_id"],
                                                         gdf_Parsed.iloc[-1]["shape_id"],
                                                         gdf_Parsed.iloc[-1]["polygon_id"]))
        print("Next id: geo:{}, shape:{}, poly:{}".format(next_i, next_shape, next_poly))

        gdf_Parsed["Type"] = type_.upper()

        # saving the result 
        file_name = file_name.split(".")[0]
        gdf_Parsed.to_csv(file_name+"_Parsed.csv", index=False)